- FAISS
- Chroma DB
- Astra DB

In [5]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [6]:
loader = TextLoader("speech.txt")
document = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=30)
texts = text_splitter.split_documents(document)
embeddings = OllamaEmbeddings(model="gemma2:2b")

db = FAISS.from_documents(texts, embeddings)

Created a chunk of size 470, which is longer than the specified 200
Created a chunk of size 347, which is longer than the specified 200
Created a chunk of size 668, which is longer than the specified 200
Created a chunk of size 982, which is longer than the specified 200
Created a chunk of size 789, which is longer than the specified 200
Created a chunk of size 355, which is longer than the specified 200


In [7]:
query = "What is the difference between HTTP and HTTPS?"

docs = db.similarity_search(query)
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'speech.txt'}, page_content="Here are some sets of questions across various categories:\nGeneral Knowledge\nWhat is the capital of France?\nWho is the CEO of Tesla?\nWhat is the largest planet in our solar system?\nWho painted the Mona Lisa?\nWhat is the chemical symbol for gold?\nTechnology\nWhat is the difference between HTTP and HTTPS?\nWhat is the latest version of Python?\nHow does blockchain technology work?\nWhat is the purpos

In [8]:
retriever = db.as_retriever()
retriever.get_relevant_documents(query)

/var/folders/fq/21nt5n9d46bgh23zzhvzdgk00000gn/T/ipykernel_38223/2506103057.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  retriever.get_relevant_documents(query)


[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'speech.txt'}, page_content="Here are some sets of questions across various categories:\nGeneral Knowledge\nWhat is the capital of France?\nWho is the CEO of Tesla?\nWhat is the largest planet in our solar system?\nWho painted the Mona Lisa?\nWhat is the chemical symbol for gold?\nTechnology\nWhat is the difference between HTTP and HTTPS?\nWhat is the latest version of Python?\nHow does blockchain technology work?\nWhat is the purpos

In [11]:
db.similarity_search_with_score(query)

[(Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
  7856.5034),
 (Document(metadata={'source': 'speech.txt'}, page_content="Here are some sets of questions across various categories:\nGeneral Knowledge\nWhat is the capital of France?\nWho is the CEO of Tesla?\nWhat is the largest planet in our solar system?\nWho painted the Mona Lisa?\nWhat is the chemical symbol for gold?\nTechnology\nWhat is the difference between HTTP and HTTPS?\nWhat is the latest version of Python?\nHow does blockchain technology work?\nWh

In [12]:
embeddings_vector = embeddings.embed_query(query)
docs = db.similarity_search_by_vector(embeddings_vector)

In [14]:
db.save_local("faiss_index")

In [16]:
new_db = FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)

In [18]:
new_db.similarity_search(query=query)

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'speech.txt'}, page_content="Here are some sets of questions across various categories:\nGeneral Knowledge\nWhat is the capital of France?\nWho is the CEO of Tesla?\nWhat is the largest planet in our solar system?\nWho painted the Mona Lisa?\nWhat is the chemical symbol for gold?\nTechnology\nWhat is the difference between HTTP and HTTPS?\nWhat is the latest version of Python?\nHow does blockchain technology work?\nWhat is the purpos

In [19]:
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [21]:
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0)
splits = text_splitter.split_documents(document)
embeddings = OllamaEmbeddings(model="gemma2:2b")

vectorDB = Chroma.from_documents(documents=splits,embedding=embeddings)

In [23]:
docs = vectorDB.similarity_search(query)
docs[0].page_content

'Who painted the Mona Lisa?\nWhat is the chemical symbol for gold?\nTechnology\nWhat is the difference between HTTP and HTTPS?\nWhat is the latest version of Python?\nHow does blockchain technology work?'

In [24]:
vectorDB = Chroma.from_documents(documents=splits,embedding=embeddings,persist_directory="./chroma")

In [26]:
db2 = Chroma(persist_directory="./chroma",embedding_function=embeddings)

In [27]:
docs = db2.similarity_search(query)

docs[0].page_content

'Who painted the Mona Lisa?\nWhat is the chemical symbol for gold?\nTechnology\nWhat is the difference between HTTP and HTTPS?\nWhat is the latest version of Python?\nHow does blockchain technology work?'